In [ ]:
from hypothesis import given, settings, example, assume
import hypothesis.strategies as stra
from collections import Counter
import unittest
from hypothesis.extra.datetime import datetimes
from hypothesis.extra.fakefactory import fake_factory
from string import ascii_lowercase
from pprint import pprint

settings.database = None

def run_all(TestClass):
    """Runs all test methods of the class"""
    suite = unittest.TestSuite()
    suite = unittest.TestLoader().loadTestsFromTestCase(TestClass)
    runner = unittest.TextTestRunner()
    runner.run(suite)

## "Example based testing"

In [ ]:
def add_one_to_elements(numbers):
    return [number + 1 for number in numbers]

def test_add_one_to_elements():
    assert add_one_to_elements([1, 0, 2]) == [2, 1, 3]
    
    
    assert add_one_to_elements([]) == []
    assert add_one_to_elements([-1]) == [0]
    assert add_one_to_elements([0.5]) == [1.5]
    
    
test_add_one_to_elements()

* What should I test? When am I finished? Thinking of examples looks like a task stupid enough for a computer.
* Unexpected data: Passing tests don't ensure correct behaviour
* If you would now cases where your program fails, you would have written it differently :-)
* Bugs which occur only when several features are used together
 * $N$ features: $N$ tests $\rightarrow$ $N$ features: $N^N$ tests

## Random generation of examples

In [ ]:
from hypothesis import given, settings, example
import hypothesis.strategies as stra
settings.database = None

list_of_floats = stra.lists(stra.floats(allow_nan=False))

@given(list_of_floats)
@settings(max_examples=5)
def test_add_one_to_elements(numbers):
    print(numbers, end='\n\n')
    assert add_one_to_elements(numbers) == list(map(lambda x: x+1, numbers))


test_add_one_to_elements()

* You don't have to waste time to construct mock data.
* Adapt strategies by chaining, mapping, filtering

In [ ]:
from hypothesis.extra.datetime import datetimes
from hypothesis.extra.fakefactory import fake_factory
from pprint import pprint

email_and_timestamp = stra.fixed_dictionaries(
    {
        'timestamp': datetimes(min_year=1900, max_year=2016).map(str),
        'name': fake_factory('name'),      
    }
)

custom_dict_strategy = stra.dictionaries(
    fake_factory('email'),
    email_and_timestamp,
    min_size=5
)

pprint(custom_dict_strategy.example())

Much more:
* Django models, Numpy arrays http://hypothesis.readthedocs.org/en/latest/extras.html
* Recursive strategies (e.g. JSON) http://hypothesis.readthedocs.org/en/latest/data.html#recursive-data

In [ ]:
def get_bytestring(string):
    return string.encode('ascii')

get_bytestring('test')

In [ ]:
@given(stra.text())
def test_get_bytestring(s):
    get_bytestring(s)

test_get_bytestring()

## Minification

In [ ]:
def add_one_to_elements(numbers):
    def _add_one(number):
        if number > 81:
            1/0
        return number + 1

    return [_add_one(number) for number in numbers]


@given(stra.lists(stra.floats()))
def test_add_one_to_elements(numbers):
    #print(numbers)
    assert add_one_to_elements(numbers) == [number + 1 for number in numbers]
    
test_add_one_to_elements()

## Properties

* Higher abstraction
* Can capture the requirements
* What properties do I want instead of examples

### Example: Sorting

In [ ]:
def quicksort(numbers):
    # TODO: there might be a bug in here
    if numbers == []:
        return []
    first, *rest = numbers # python3 same as
                           # first, rest = numbers[0], numbers[1:]
    smaller = [element for element in rest if element < first] # should be element <= first
    bigger = [element for element in rest if element > first]
    return quicksort(smaller) + [first] + quicksort(bigger)

quicksort([5,3,10,-100])

In [ ]:
from collections import Counter

Counter("aasdlfjsld")

In [ ]:
from collections import Counter
import unittest

class TestSorting(unittest.TestCase):

    @given(stra.lists(stra.integers()))
    def test_ordering(self, numbers):
        """Every element has to be smaller or equal than the following element."""
        sorted_numbers = quicksort(numbers)
        order_correct = (
            element <= next_element for element, next_element in zip(sorted_numbers[:-1], sorted_numbers[1:])
        )
        self.assertTrue(all(order_correct))

    @given(stra.lists(stra.integers()))
    def test_permutation(self, numbers):
        """The elements have to stay the same."""
        self.assertEqual(Counter(numbers), Counter(quicksort(numbers)))
        
    @given(stra.lists(stra.integers()))
    def test_idempotence(self, numbers):
        """Result does not change on multiple application of the function."""
        self.assertEqual(quicksort(numbers), quicksort(quicksort(numbers)))


run_all(TestSorting)

### Sidenote: Complex or complicated?

In [ ]:
# From Problem Solving with Algorithms and Data Structures, Chapter Sorting and Searching
# http://interactivepython.org/runestone/static/pythonds/SortSearch/TheQuickSort.html

def quickSort(alist):
   quickSortHelper(alist,0,len(alist)-1)

def quickSortHelper(alist,first,last):
   if first<last:

       splitpoint = partition(alist,first,last)

       quickSortHelper(alist,first,splitpoint-1)
       quickSortHelper(alist,splitpoint+1,last)


def partition(alist,first,last):
   pivotvalue = alist[first]

   leftmark = first+1
   rightmark = last

   done = False
   while not done:

       while leftmark <= rightmark and alist[leftmark] <= pivotvalue:
           leftmark = leftmark + 1

       while alist[rightmark] >= pivotvalue and rightmark >= leftmark:
           rightmark = rightmark -1

       if rightmark < leftmark:
           done = True
       else:
           temp = alist[leftmark]
           alist[leftmark] = alist[rightmark]
           alist[rightmark] = temp

   temp = alist[first]
   alist[first] = alist[rightmark]
   alist[rightmark] = temp


   return rightmark

a = [1,4,3,2,2,2,2,2,2]
quickSort(a)
print(a)

## Side effects

In [ ]:
def test_me():
    input_strings = []
    while len(input_strings) < 4:
        input_string = input() + '!'
        input_strings.append(input_string)
    return input_strings

In [ ]:
test_me()

In [ ]:
def test_me():
    return test_me_pure(get_inputs())

def get_inputs():
    while True:
        yield input()
    
def test_me_pure(inputs):
    input_strings = []
    for i in range(4):
        input_string = next(inputs) + '!'
        input_strings.append(input_string)
    return input_strings
    # Same in one line
    # return [next(inputs) + "!" for _ in range(4)]


In [ ]:
test_me()

In [ ]:
@given(stra.streaming(stra.text()))
def test_test_me_pure(inputs):
    result = test_me_pure(iter(inputs))
    
    # length is 4
    assert len(result) == 4
    
    # last character is "!"
    for element in result:
        assert element.endswith('!')

    # inverse (stripping of the last character) gets back the original input
    assert list(inputs[:4]) == [element[:-1] for element in result]

test_test_me_pure()

## Pitfall: Testing the wrong distribution

In [ ]:
def split(delimiter, string):
    # TODO: there might be a bug in here
    
    splitted_strings = []
    
    while(True):
        next_delimiter = string.find(delimiter)
        
        if next_delimiter >= 0:
            splitted_strings.append(string[:next_delimiter])
            string = string[next_delimiter+1:]
        else:
            if string:
                splitted_strings.append(string)
            break
            
    return splitted_strings

split('@', 'fuu@bar')

In [ ]:
from string import ascii_lowercase

class TestSplit(unittest.TestCase):
    
    @given(
        delimiter=stra.text(alphabet=ascii_lowercase, min_size=1, max_size=1),
        string=stra.text(alphabet=ascii_lowercase)
    )
    @settings(max_examples=10)
    def test_split(self, delimiter, string):
        print('Teststring: ', string)
        print('Delimiter: ', delimiter)
        splitted_strings = split(delimiter,string)
        self.assertEqual(string, delimiter.join(splitted_strings))

run_all(TestSplit)

* data.draw strategy

In [ ]:
class TestSplit(unittest.TestCase):
    
    @given(stra.data())
    @settings(max_examples=10)
    def test_split(self, data):
        string = data.draw(stra.text(min_size=1, alphabet=ascii_lowercase))
        print('Teststring: ', string)
        delimiter = data.draw(stra.sampled_from(string))
        print('Delimiter: ', delimiter)
        splitted_strings = split(delimiter,string)
        self.assertEqual(string, delimiter.join(splitted_strings))

run_all(TestSplit)

In [ ]:
splitted = split('a', 'a')
splitted
'a'.join(splitted)
'a'.split('a')

## Meta: Counter examples expose errors in your thinking

**The Experiment**
  * We monitor the bugs / lines of code of two programmers over two days
  * If one programmer has been "better" on both days he should have less bugs / lines of codes in total
  * Right?

In [ ]:
from hypothesis import assume

def _get_ratios(bugs, lines_of_code):
    return [
        bugs_that_day / loc_that_day for bugs_that_day, loc_that_day in zip(bugs, lines_of_code)
    ]

# We will throw most of these examples away, could be made more efficient
bugs_strategy = stra.lists(stra.integers(min_value=0), min_size=2, max_size=2)
loc_strategy = stra.lists(stra.integers(min_value=1), min_size=2, max_size=2)

class TestBugRatios(unittest.TestCase):
    @given(
        bugs_A=bugs_strategy,
        lines_of_code_A=loc_strategy,
        
        bugs_B=bugs_strategy,
        lines_of_code_B=loc_strategy,
    )
    def test_bug_ratio(self, bugs_A, lines_of_code_A, bugs_B, lines_of_code_B):
        """If the bug ratio has been less for programmer A in the individual days, the total ratio has to be also less."""
        
        ratios_A = _get_ratios(bugs_A, lines_of_code_A)
        ratios_B = _get_ratios(bugs_B, lines_of_code_B)
        
        both_ratios_smaller = (ratios_A[0] < ratios_B[0]) & (ratios_A[1] < ratios_B[1])
        assume(both_ratios_smaller)
        
        total_ratio_A = sum(bugs_A) / sum(lines_of_code_A)
        total_ratio_B = sum(bugs_B) / sum(lines_of_code_B)
        self.assertLess(
            total_ratio_A, total_ratio_B,
            msg='even though the individual ratios are {} and {}'.format(ratios_A, ratios_B)
        )
        
run_all(TestBugRatios)

See https://en.wikipedia.org/wiki/Simpson's_paradox

## The next level: Stateful testing

Debugging is a search problem, can it be automated?

In [ ]:
import turtle
t = turtle.Turtle(shape='turtle')

for i in range(150):
    t.forward(10)
    t.left(0.1 * i)


win = turtle.Screen()
win.exitonclick()

In [ ]:
from hypothesis import strategies as stra
from hypothesis.stateful import RuleBasedStateMachine, rule
import unittest
import turtle
from scipy.spatial.distance import euclidean

from hypothesis import settings, Verbosity
settings.database = None

def run_all(TestClass):
    """Runs all test methods of the class"""
    suite = unittest.TestSuite()
    suite = unittest.TestLoader().loadTestsFromTestCase(TestClass)
    runner = unittest.TextTestRunner()
    runner.run(suite)
    

class FailTurtle(turtle.Turtle):

    def forward(self, distance):
        if hasattr(self, 'fail_point'):
            if euclidean(self.position(), self.fail_point) < 10:
                1/0
        super().forward(distance)


def get_turtle(fail_point):
    t = FailTurtle(shape='turtle')
    t.color('black', 'green')
    t.up()
    t.goto(fail_point)
    t.dot(10, 'red')
    t.goto(0, 0)
    t.down()
    t.fail_point = fail_point
    return t
        
class TurtleMachine(RuleBasedStateMachine):
    
    def __init__(self):
        self.t = get_turtle((20, 20))
        self.bounding_size = 80

    @rule(turn=stra.integers(min_value=-90, max_value=90), forward=stra.integers(min_value=10, max_value=10,))
    def move(self, turn, forward):
        self.t.right(turn)
        self.t.forward(forward)


TestTurtle = TurtleMachine.TestCase
TestTurtle.settings = settings(max_examples=10, stateful_step_count=35, max_shrinks=3)
run_all(TestTurtle)
win = turtle.Screen()
win.exitonclick()

## Links/Acknowledgements

* http://hypothesis.readthedocs.org/
* [John Hughes - Testing the Hard Stuff and Staying Sane](https://www.youtube.com/watch?v=zi0rHwfiX1Q)
* [Moritz Gronbach - What's the fuzz all about? Randomized data generation for robust unit testing](https://www.youtube.com/watch?v=ABnqAnhonDk)
* http://fsharpforfunandprofit.com/posts/property-based-testing-2/
* https://wiki.haskell.org/Introduction_to_QuickCheck1
* http://book.realworldhaskell.org/read/testing-and-quality-assurance.html
* https://www.schoolofhaskell.com/user/pbv/an-introduction-to-quickcheck-testing